#Introduction to Protein Sequence Encoding and Functional Classification

#### ProtVec: The Publication
The **ProtVec** model was introduced in the publication by Asgari and Mofrad (https://arxiv.org/abs/1503.05140). The study demonstrated that embeddings derived from protein sequences could enhance tasks like protein classification, functional prediction, and anomaly detection. By representing proteins in numerical form, ProtVec enables the application of machine learning techniques for bioinformatics research.

### Why Use Encoding Instead of Raw Sequences?
Proteins, composed of amino acid sequences, are essential biological macromolecules that carry out diverse cellular functions. While these sequences provide fundamental information, analyzing raw sequences directly poses several challenges:
- **Dimensionality**: Protein sequences vary in length, making direct comparisons computationally inefficient.
- **Lack of Numerical Representation**: Machine learning models require numerical input, but raw sequences are textual.
- **Contextual Understanding**: Proteins operate based on structural and functional motifs, which are difficult to infer from linear sequences alone.

To overcome these limitations, encoding techniques transform sequences into numerical representations that capture meaningful biological patterns. These encoded forms allow computational models to classify and predict protein functions efficiently.

### Protein Function and InterPro
Proteins are categorized into families based on sequence similarities, structural features, and functional domains. **InterPro** is a comprehensive database that integrates multiple protein signature databases to classify proteins and predict their functions. It helps:
- Identify conserved protein domains.
- Classify proteins into families based on evolutionary relationships.
- Infer potential functions based on domain architecture.

InterPro plays a crucial role in annotating newly discovered proteins, enabling researchers to associate unknown sequences with known biological functions.

### Importance of Functional Classification
Functional classification of proteins is fundamental in bioinformatics and molecular biology. It allows:
- **Understanding Biological Systems**: Identifying how proteins interact within cellular pathways.
- **Drug Discovery**: Recognizing target proteins for therapeutic development.
- **Evolutionary Studies**: Analyzing evolutionary relationships across species.

By grouping proteins into families, scientists can predict the function of unknown sequences, facilitating advances in medicine, agriculture, and synthetic biology.

### ProtVec: Encoding Protein Sequences for Computational Analysis
**ProtVec** is an unsupervised learning method inspired by **Word2Vec**, designed to encode protein sequences into fixed-length vector representations. It utilizes **3-gram** embeddings, where each sequence is broken into overlapping triplets of amino acids. These triplets are then used to train a neural network, generating dense vector representations that capture sequence similarities and evolutionary signals.

### Task Description
Your task is to use **ProtVec** embeddings to classify protein sequences into families based on the dataset from the ProtVec publication (http://dx.doi.org/10.7910/DVN/JMFHTN). Follow these steps:
1. **Preprocess the Protein Sequences**: Convert raw sequences into ProtVec embeddings.
2. **Select 5 biggest families**: This will simplify the training process
2. **Train a Classification Model**: Use the embeddings to train a machine learning classifier.
3. **Evaluate Model Performance**: Assess classification accuracy and interpret the results.

### Expected Outcome
By completing this task, you will:
- Gain experience in transforming protein sequences into numerical vectors.
- Understand the role of functional classification in bioinformatics.
- Apply computational techniques to biological data, bridging biology and informatics.

Good luck, and enjoy the challenge!

In [69]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# load data to classify from the original paper
sequences = pd.read_csv("family_classification_sequences.csv") 
sequences.head()

# load metadata for each sequence
metadata = pd.read_excel("family_classification_metadata.xlsx") 

# combine data and metadata
train = pd.concat([sequences, metadata['Family ID']], axis=1) 
train.head()

c:\Users\nelia\OneDrive\Рабочий стол\semester 2\DeepLife\.venv_deeplife\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sequences,Family ID
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,Pox_VLTF3
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,DUF230
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...,US22
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...,DUF3627
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...,DUF2738


In [36]:
# select 5 greatest families
top5 = train['Family ID'].value_counts().head(5).index
train = train[train['Family ID'].isin(top5)] 

In [74]:
# load pretrained embeddings for each 3-gram
pre_trained = pd.read_csv("protVec_100d_3grams.csv", delimiter='\t') 
# convert to dictionary with word as key and embedding as numpy array
embeddings = {row[0]: np.array(row[1:]) for row in pre_trained.values}
print(embeddings['AAA'])

[-0.17406 -0.095756 0.059515 0.039673 -0.375934 -0.115415 0.090725
 0.173422 0.29252 0.190375 0.094091 -0.197482 -0.135202 0.075521 0.110771
 0.047909 -0.391934 0.073548 0.103868 -0.045924 -0.009534 0.055659
 -0.000308 0.215941 0.084476 0.061573 0.128139 0.184247 -0.100091
 -0.126661 -0.005728 -0.038272 0.180597 -0.15531 0.056232 -0.005925
 -0.085381 -0.056921 -0.04552 0.265116 0.090221 -0.209879 0.205381
 0.023679 -0.092939 0.072767 -0.105107 0.011112 -0.160518 0.042627 0.15123
 -0.162708 -0.083479 -0.146657 0.091332 0.109579 -0.101678 0.091198
 0.005512 0.047318 0.078108 0.203824 -0.100126 0.294703 -0.158841 0.029333
 0.078265 0.018524 0.117082 0.212755 -0.171555 0.029421 0.149264 0.046599
 -0.184111 0.294123 -0.101497 -0.030123 -0.009826 0.007835 -0.106508
 -0.166202 -0.024748 -0.090856 0.056977 0.047644 0.018618 -0.034376
 0.087013 -0.278817 0.244482 0.015974 0.012903 0.137528 0.13814 0.005474
 0.070719 -0.164084 -0.179274 0.184899]


In [85]:
# convert raw sequences to 3-grams and then to embeddings
# in the original paper, they used 100-dimensional embeddings for each 3-gram, which were then summed to get a single 100-dimensional embedding for each sequence
train_embed = train.copy()
cols = pre_trained.columns[1:]
train_embed[cols] = 0
print(train_embed.head())
k = 0
for index in train.index:
    seq = train.loc[index, 'Sequences']
    vectors = [embeddings[seq[i:i+3]] for i in range(len(seq)-2)
                if seq[i:i+3] in embeddings]
    if vectors:
        train_embed.loc[index, cols] = np.sum(vectors, axis=0)
    k += 1
    if k % 100 == 0:
        print(k)

train_embed.head()



C:\Users\nelia\AppData\Local\Temp\ipykernel_22404\3263894789.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_embed[cols] = 0
C:\Users\nelia\AppData\Local\Temp\ipykernel_22404\3263894789.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-74.7053879999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_embed.loc[index, cols] = np.sum(vectors, axis=0)
C:\Users\nelia\AppData\Local\Temp\ipykernel_22404\3263894789.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-11.096257999999994' has dtype incompatible with int64, please explicitly c

                                             Sequences   Family ID  d1  d2  \
5    MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...  Helicase_C   0   0   
69   MSTILEKISAIESEMARTQKNKATSAHLGLLKAKLAKLRRELISPK...    MMR_HSR1   0   0   
86   MADAMNELCNLTQHLQVDDDQLSNLKLKNGYSLFPHQEKVMLWMKY...  Helicase_C   0   0   
697  MDLRATSSNDSNATSGYSDTAAVDWDEGENATGSGSLPDPELSYQI...       7tm_1   0   0   
698  MEGTNNTTGWTHFDSTSNRTSKSFDEEVKLSYQVVTSFLLGALILC...       7tm_1   0   0   

     d3  d4  d5  d6  d7  d8  ...  d91  d92  d93  d94  d95  d96  d97  d98  d99  \
5     0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0    0   
69    0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0    0   
86    0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0    0   
697   0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0    0   
698   0   0   0   0   0   0  ...    0    0    0    0    0    0    0    0    0   

     d100  
5       0  
69      0  
86      

,Sequences,Family ID,d1,d2,d3,d4,d5,d6,d7,d8,...,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100
5,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...,Helicase_C,-74.705388,-11.096258,13.424886,-57.402324,-2.046353,-0.773480,18.072000,-4.549276,...,21.117913,-36.998461,-21.092598,18.451578,25.563415,12.747790,-15.781115,-44.443070,-33.764278,65.268231
69,MSTILEKISAIESEMARTQKNKATSAHLGLLKAKLAKLRRELISPK...,MMR_HSR1,-24.020902,0.515459,5.917351,-26.775791,-3.168579,-3.808098,13.255935,-7.840464,...,-0.155092,-7.145848,0.046244,2.233395,9.085530,0.838310,-4.192336,-15.318800,-5.456624,28.549326
86,MADAMNELCNLTQHLQVDDDQLSNLKLKNGYSLFPHQEKVMLWMKY...,Helicase_C,-35.480909,3.713599,-4.594302,-44.900905,6.617420,-9.574256,22.063496,-18.308511,...,-3.300902,-4.315914,-7.462813,-10.222807,18.310813,3.528239,-13.369621,-32.378225,-7.496086,52.301561
697,MDLRATSSNDSNATSGYSDTAAVDWDEGENATGSGSLPDPELSYQI...,7tm_1,-27.434558,-6.760551,-5.059816,-35.227506,0.197853,-6.468589,8.024180,-10.135642,...,9.981873,-14.372620,-15.233308,-5.080663,8.947220,0.305354,-7.873150,-26.515991,-14.979560,38.721649
698,MEGTNNTTGWTHFDSTSNRTSKSFDEEVKLSYQVVTSFLLGALILC...,7tm_1,-27.587421,-4.400698,-5.767396,-35.986144,1.006702,-3.553814,10.417845,-14.038094,...,6.766127,-10.612250,-17.229227,-3.855176,9.857280,0.078984,-9.563518,-24.914619,-11.680858,39.845652


In [86]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(train_embed[cols], train_embed['Family ID'], test_size=0.2, random_state=42)


In [87]:
# train random forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# evaluate model
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Model accuracy is 0.97 using simple model and only 5 families. 
# however, it is still a good result. Using pretrained embeddings is an effective way to represent sequences for classification.

Accuracy: 0.97
